<a href="https://colab.research.google.com/github/manualrg/dslab-nlp-dl/blob/master/08_dlnlp_vecsearch_ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embeddings y busqueda vectorial

In [ ]:
import typing

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 60.5 MB/s eta 0:00:00


# ¿Qué son los embeddings?
Los embeddings son representaciones numéricas densas (vectores) de datos como texto.

Palabras o frases similares en significado estarán cerca en el espacio vectorial.

Modelos como Word2Vec, GloVe y BERT generan diferentes tipos de embeddings.

Nosotros usaremos Sentence-BERT (SBERT) porque genera vectores semánticos de oraciones directamente.

Especificamente, los generados por [all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2)

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

sentences = [
    "El clima está soleado hoy.",
    "Hoy hace mucho sol y calor.",
    "Estoy aprendiendo machine learning.",
    "El perro ladra fuerte.",
    "La inteligencia artificial está avanzando rápidamente."
]

embeddings = model.encode(sentences,
                          normalize_embeddings=True  #
                          )


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

vec_q = embeddings[0]
veq_k = embeddings

similarities = cosine_similarity([vec_q], veq_k)
for i, score in enumerate(similarities[0]):
    print(f"→ Sim('{sentences[0]}', '{sentences[i]}') = {score:.2f}")

→ Sim('El clima está soleado hoy.', 'El clima está soleado hoy.') = 1.00
→ Sim('El clima está soleado hoy.', 'Hoy hace mucho sol y calor.') = 0.51
→ Sim('El clima está soleado hoy.', 'Estoy aprendiendo machine learning.') = 0.31
→ Sim('El clima está soleado hoy.', 'El perro ladra fuerte.') = 0.60
→ Sim('El clima está soleado hoy.', 'La inteligencia artificial está avanzando rápidamente.') = 0.40


# Parte 5: Búsqueda semántica con FAISS

Facebook AI Similarity Search (FAISS) es una libreria para realizar busqueda vectorial


In [ ]:
import faiss
import numpy as np


# Creacion de un indice con el corpus
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Usamos inner product ≈ cosine similarity con vectores normalizados
index.add(embeddings)


In [ ]:
# busqueda del documento mas parecido a vec_q dentro del indice vec_k

text = "El clima está soleado hoy."
k= 3
vec_q = model.encode([text], normalize_embeddings=True)
D, I = index.search(vec_q, k=k)

for i_ret, dist in zip(I[0],D[0]):
    print(f"→ {sentences[i_ret]}, {dist}")

→ El clima está soleado hoy., 1.0
→ El perro ladra fuerte., 0.5976425409317017
→ Hoy hace mucho sol y calor., 0.5134171843528748


# Ejercicio

In [ ]:
corpus = [
    "Smartphone de gama alta con pantalla OLED de 6.7 pulgadas, procesador de ocho núcleos, 128GB de almacenamiento, triple cámara trasera de 50MP y batería de larga duración. Diseño elegante en color negro. Ideal para fotografía y gaming.",
    "Auriculares inalámbricos con cancelación de ruido activa, sonido de alta fidelidad, hasta 30 horas de batería y cómodas almohadillas de espuma viscoelástica. Incluye estuche de carga rápido. Perfectos para viajes y trabajo.",
    "Smartwatch deportivo con GPS integrado, monitor de ritmo cardíaco continuo, seguimiento de sueño, resistencia al agua de 50 metros y notificaciones inteligentes. Correa de silicona azul. Compatible con Android e iOS.",
    "Cafetera espresso automática con molinillo integrado, 15 bares de presión, depósito de agua de 1.8 litros y funciones programables. Prepara capuccinos y lattes con facilidad. Acabado en acero inoxidable.",
    "Aspiradora robot inteligente con mapeo láser, succión potente, control por aplicación móvil y compatibilidad con asistentes de voz. Ideal para hogares con mascotas. Incluye base de autocarga.",
    "Televisor 4K UHD de 55 pulgadas con tecnología HDR, Smart TV integrado, tres puertos HDMI y dos puertos USB. Ofrece colores vibrantes y negros profundos. Marco delgado y diseño moderno.",
    "Laptop ultradelgada con procesador Intel Core i7, 16GB de RAM, SSD de 512GB, pantalla Full HD de 14 pulgadas y teclado retroiluminado. Perfecta para productividad y movilidad. Carcasa de aluminio.",
    "Altavoz Bluetooth portátil con sonido de 360 grados, resistencia al agua IPX7, hasta 12 horas de reproducción y función de emparejamiento estéreo. Diseño compacto y robusto en color rojo.",
    "Licuadora de alto rendimiento con motor de 1200W, jarra de vidrio de 1.5 litros, seis cuchillas de acero inoxidable y múltiples velocidades. Ideal para smoothies, sopas y salsas.",
    "Horno de convección eléctrico de 60 litros con control de temperatura digital, temporizador, funciones de grill y descongelación. Perfecto para hornear y asar uniformemente.",
    "Bicicleta de montaña de 29 pulgadas con cuadro de aluminio ligero, suspensión delantera, 21 velocidades y frenos de disco hidráulicos. Ideal para senderos y rutas off-road.",
    "Cámara digital sin espejo con sensor APS-C de 24MP, grabación de video 4K, enfoque automático rápido y pantalla táctil abatible. Incluye lente de kit 18-55mm. Compacta y versátil.",
    "Router Wi-Fi 6 de doble banda con velocidades de hasta 3000 Mbps, cuatro antenas externas y tecnología OFDMA. Mejora la conexión en hogares grandes con múltiples dispositivos.",
    "Impresora multifunción de inyección de tinta con impresión, copia y escaneo, conectividad Wi-Fi, impresión a doble cara automática y alimentador automático de documentos. Económica y eficiente.",
    "Máquina de coser computarizada con 100 puntadas, enhebrador automático, pantalla LCD y control de velocidad. Ideal para proyectos de costura creativa y reparaciones.",
    "Proyector Full HD portátil con brillo de 3000 lúmenes, corrección trapezoidal automática, altavoces integrados y conectividad HDMI/USB. Perfecto para cine en casa y presentaciones.",
    "Freidora de aire caliente de 5.5 litros con 8 programas preestablecidos, control de temperatura digital y cesta antiadherente. Cocina de forma saludable con poco o nada de aceite.",
    "Monitor gaming de 27 pulgadas con resolución QHD, tasa de refresco de 144Hz, tiempo de respuesta de 1ms y tecnología FreeSync. Imágenes fluidas y detalladas para una experiencia inmersiva.",
    "Set de ollas y sartenes de acero inoxidable de 10 piezas con bases encapsuladas para una distribución uniforme del calor y tapas de vidrio templado. Compatibles con inducción.",
    "Robot de cocina inteligente con pantalla táctil, más de 500 recetas guiadas, funciones de cocción, mezcla y amasado. Simplifica la preparación de comidas. Conectividad Wi-Fi."
]

In [ ]:
vec_qs = [
    "Necesito unos auriculares para viajar que tengan buena cancelación de ruido y que la batería dure mucho.",
    "Estoy buscando un ordenador portátil fino y ligero para trabajar, con buena RAM y almacenamiento SSD.",
    "¿Qué electrodoméstico de cocina me recomiendas para hacer smoothies y sopas?"
]